In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Rucurrent Neural Network for forecasting bitcoin price 

Created on Sun Jun 14 2022

@author: chenqian
"""

# univariate multi-step vector-output stacked lstm example
import csv
import pickle
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
import numpy as np
import pandas as pd
# multi-step data preparation
from numpy import array


In [ ]:
dfv = pd.read_csv('Full_data.csv')        
df_n = pd.read_csv('WHO-COVID-19-global-data.csv') 
df_n.info()

In [4]:
# ETL process(country Canada, United States of America, Spain, 2021-01-018 to 2022-02-27)
df = df_n[df_n['Country'].isin(['Spain','Canada','United States of America'])]
df = df[df['New_cases'].notnull()]
df = df[df['Cumulative_cases'].notnull()]
df = df[df['New_deaths'].notnull()]
df = df[df['Cumulative_deaths'].notnull()]
df = df.reset_index(drop = True)
df_c = df[(df['Country'] == 'Canada') & (df['Date_reported'] >= '2021-01-18') & (df['Date_reported'] <= '2022-02-27')]
df_s = df[(df['Country'] == 'Spain') & (df['Date_reported'] >= '2021-01-18') & (df['Date_reported'] <= '2022-02-27')]
df_u = df[(df['Country_code'] == 'US') & (df['Date_reported'] >= '2021-01-18') & (df['Date_reported'] <= '2022-02-27')]
df_c = df_c.reset_index(drop = True)
df_s = df_s.reset_index(drop = True)
df_u = df_u.reset_index(drop = True)


df_c['week'] = pd.DataFrame([j for j in range(int(df_c.shape[0]/7)) for i in range(7)])
df_s['week'] = pd.DataFrame([j for j in range(int(df_c.shape[0]/7)) for i in range(7)])
df_u['week'] = pd.DataFrame([j for j in range(int(df_c.shape[0]/7)) for i in range(7)])


df_c['transmission_rate'] = df_c['New_cases']/df_c['Cumulative_cases']
df_s['transmission_rate'] = df_s['New_cases']/df_c['Cumulative_cases']
df_u['transmission_rate'] = df_u['New_cases']/df_c['Cumulative_cases']

df_c['case_fatality_rate'] = df_c['Cumulative_deaths']/df_c['Cumulative_cases']
df_s['case_fatality_rate'] = df_s['Cumulative_deaths']/df_c['Cumulative_cases']
df_u['case_fatality_rate'] = df_u['Cumulative_deaths']/df_c['Cumulative_cases']
df_c = df_c.groupby('week').mean().reset_index()
df_s = df_s.groupby('week').mean().reset_index()
df_u = df_u.groupby('week').mean().reset_index()
df_c['Country'] = 'Canada'
df_s['Country'] = 'Spain'
df_u['Country'] = 'United States'

# useful vaccinated data
dfv = dfv[dfv['location'].isin(['Canada','United States', 'Spain'])][['location', 'date','people_fully_vaccinated']]

dfv_c = dfv[(dfv['location'] == 'Canada') & (dfv['date'] >= '2021-01-18') & (dfv['date'] <= '2022-02-27')]
dfv_u = dfv[(dfv['location'] == 'United States') & (dfv['date'] >= '2021-01-18') & (dfv['date'] <= '2022-02-27')]
dfv_s = dfv[(dfv['location'] == 'Spain') & (dfv['date'] >= '2021-01-18') & (dfv['date'] <= '2022-02-27')]
dfv_c = dfv_c.reset_index(drop = True)
dfv_s = dfv_s.reset_index(drop = True)
dfv_u = dfv_u.reset_index(drop = True)
dfv_c['week'] = pd.DataFrame([j for j in range(int(dfv_c.shape[0]/7)) for i in range(7)])
dfv_u['week'] = pd.DataFrame([j for j in range(int(dfv_u.shape[0]/7)) for i in range(7)])
dfv_s['week'] = pd.DataFrame([j for j in range(int(dfv_s.shape[0]/7)) for i in range(7)])
dfv_c = dfv_c.groupby('week').max().reset_index()
dfv_u = dfv_u.groupby('week').max().reset_index()
dfv_s = dfv_s.groupby('week').max().reset_index()

dfv_c['people_fully_vaccinated'] = dfv_c['people_fully_vaccinated']/sum(dfv_c['people_fully_vaccinated']**2)**(1/2)
dfv_s['people_fully_vaccinated'] = dfv_s['people_fully_vaccinated']/sum(dfv_s['people_fully_vaccinated']**2)**(1/2)
dfv_u['people_fully_vaccinated'] = dfv_u['people_fully_vaccinated']/sum(dfv_u['people_fully_vaccinated']**2)**(1/2)

df_c['people_fully_vaccinated'] = dfv_c['people_fully_vaccinated']
df_s['people_fully_vaccinated'] = dfv_s['people_fully_vaccinated']
df_u['people_fully_vaccinated'] = dfv_u['people_fully_vaccinated']

df_auxa = pd.concat([df_u,df_s])
df_all = pd.concat([df_auxa,df_c])
df_all
df_all.to_csv('result_table.csv',index=False)


In [ ]:
# ETL process(country Canada, United States of America, Spain, 2021-01-018 to 2022-02-27)
df = df_n[df_n['Country'].isin(['Spain','Canada','United States of America'])]
df = df[df['New_cases'].notnull()]
df = df[df['Cumulative_cases'].notnull()]
df = df[df['New_deaths'].notnull()]
df = df[df['Cumulative_deaths'].notnull()]
df = df.reset_index(drop = True)
df_c = df[(df['Country'] == 'Canada') & (df['Date_reported'] >= '2021-01-18') & (df['Date_reported'] <= '2022-02-27')]
df_s = df[(df['Country'] == 'Spain') & (df['Date_reported'] >= '2021-01-18') & (df['Date_reported'] <= '2022-02-27')]
df_u = df[(df['Country_code'] == 'US') & (df['Date_reported'] >= '2021-01-18') & (df['Date_reported'] <= '2022-02-27')]
df_c = df_c.reset_index(drop = True)
df_s = df_s.reset_index(drop = True)
df_u = df_u.reset_index(drop = True)


df_c['week'] = pd.DataFrame([j for j in range(int(df_c.shape[0]/7)) for i in range(7)])
df_s['week'] = pd.DataFrame([j for j in range(int(df_c.shape[0]/7)) for i in range(7)])
df_u['week'] = pd.DataFrame([j for j in range(int(df_c.shape[0]/7)) for i in range(7)])


df_c['transmission_rate'] = df_c['New_cases']/df_c['Cumulative_cases']*10
df_s['transmission_rate'] = df_s['New_cases']/df_c['Cumulative_cases']*10
df_u['transmission_rate'] = df_u['New_cases']/df_c['Cumulative_cases']

df_c['case_fatality_rate'] = df_c['Cumulative_deaths']/df_c['Cumulative_cases']*10
df_s['case_fatality_rate'] = df_s['Cumulative_deaths']/df_c['Cumulative_cases']*10
df_u['case_fatality_rate'] = df_u['Cumulative_deaths']/df_c['Cumulative_cases']
df_c = df_c.groupby('week').mean().reset_index()
df_s = df_s.groupby('week').mean().reset_index()
df_u = df_u.groupby('week').mean().reset_index()
df_c['Country'] = 'Canada'
df_s['Country'] = 'Spain'
df_u['Country'] = 'United States'

# useful vaccinated data
dfv = dfv[dfv['location'].isin(['Canada','United States', 'Spain'])][['location', 'date','people_fully_vaccinated']]

dfv_c = dfv[(dfv['location'] == 'Canada') & (dfv['date'] >= '2021-01-18') & (dfv['date'] <= '2022-02-27')]
dfv_u = dfv[(dfv['location'] == 'United States') & (dfv['date'] >= '2021-01-18') & (dfv['date'] <= '2022-02-27')]
dfv_s = dfv[(dfv['location'] == 'Spain') & (dfv['date'] >= '2021-01-18') & (dfv['date'] <= '2022-02-27')]
dfv_c = dfv_c.reset_index(drop = True)
dfv_s = dfv_s.reset_index(drop = True)
dfv_u = dfv_u.reset_index(drop = True)
dfv_c['week'] = pd.DataFrame([j for j in range(int(dfv_c.shape[0]/7)) for i in range(7)])
dfv_u['week'] = pd.DataFrame([j for j in range(int(dfv_u.shape[0]/7)) for i in range(7)])
dfv_s['week'] = pd.DataFrame([j for j in range(int(dfv_s.shape[0]/7)) for i in range(7)])
dfv_c = dfv_c.groupby('week').max().reset_index()
dfv_u = dfv_u.groupby('week').max().reset_index()
dfv_s = dfv_s.groupby('week').max().reset_index()

dfv_c['people_fully_vaccinated'] = dfv_c['people_fully_vaccinated']/sum(dfv_c['people_fully_vaccinated']**2)**(1/2)
dfv_s['people_fully_vaccinated'] = dfv_s['people_fully_vaccinated']/sum(dfv_s['people_fully_vaccinated']**2)**(1/2)
dfv_u['people_fully_vaccinated'] = dfv_u['people_fully_vaccinated']/sum(dfv_u['people_fully_vaccinated']**2)**(1/2)

df_c['people_fully_vaccinated'] = dfv_c['people_fully_vaccinated']*10
df_s['people_fully_vaccinated'] = dfv_s['people_fully_vaccinated']*10
df_u['people_fully_vaccinated'] = dfv_u['people_fully_vaccinated']

df_auxa = pd.concat([df_u,df_s])
df_all = pd.concat([df_auxa,df_c])
df_all


In [6]:
def count_endpoint(pre):
    aux = 0
    for i in y_predend:
        if i > 0:
            aux = aux + 1
        else:
            break
    return aux         

def predict(model, x_Input,cfr_input, pfv_input, N_steps_in, N_features, num_predi):
    result = []
    for i in range(num_predi):
        data = array(x_Input)
        datau = np.append(data,  array(cfr_input))
        datau = np.append(datau, array(pfv_input))
        datau = datau.reshape((1, N_steps_in, N_features))
        predicted = model.predict(datau, verbose=0)
        result.append(predicted[0][0])
        x_Input = x_Input[1:]
        x_Input.append(predicted[0][0])
        datau = []
    return result

def predict_end_u(model, x_Input, N_steps_in, N_features, num_predi):
    result = []
    for i in range(num_predi):
        data = array(x_Input)
        datau = np.append(data,  array([[0.287071,0.287071]]))
        datau = np.append(datau, array([[0.173468,0.173468]]))
        datau = datau.reshape((1, N_steps_in, N_features))
        predicted = model.predict(datau, verbose=0)
        result.append(predicted[0][0])
        x_Input = x_Input[1:]
        x_Input.append(predicted[0][0])
        datau = []
    return result

def predict_end_s(model, x_Input, N_steps_in, N_features, num_predi):
    result = []
    for i in range(num_predi):
        data = array(x_Input)
        datau = np.append(data,  array([[0.30535,0.30535]]))
        datau = np.append(datau, array([[1.85592,1.85592]]))
        datau = datau.reshape((1, N_steps_in, N_features))
        predicted = model.predict(datau, verbose=0)
        result.append(predicted[0][0])
        x_Input = x_Input[1:]
        x_Input.append(predicted[0][0])
        datau = []
    return result

def predict_end_c(model, x_Input, N_steps_in, N_features, num_predi):
    result = []
    for i in range(num_predi):
        data = array(x_Input)
        datau = np.append(data,  array([[0.111194,0.111194]]))
        datau = np.append(datau, array([[1.92078,1.92078]]))
        datau = datau.reshape((1, N_steps_in, N_features))
        predicted = model.predict(datau, verbose=0)
        result.append(predicted[0][0])
        x_Input = x_Input[1:]
        x_Input.append(predicted[0][0])
        datau = []
    return result


def plotResults(yhat, y, num_obser):
    raw_seq_x = range(1, num_obser + 1)
    raw_seq_y = y[0:num_obser]
    raw_seq_pre = yhat[0:num_obser]
    plt.plot(raw_seq_x, raw_seq_y)
    plt.plot(raw_seq_x, raw_seq_pre)
    plt.show()
    
    
# split a univariate sequence into samples
def split_sequence(sequence,n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):# find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return array(X), array(y)

# split a univariate sequence into train data and test data
def train_test_sep(sequence, n):
    train_data, test_data = list(), list()
    train_data = sequence.iloc[0:(len(sequence)-n)]
    test_data  = sequence.iloc[(len(sequence) -n):len(sequence)]
    return train_data, test_data

# Root Mean Squared Error 
def root_mean_squared_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return (np.mean((y_true - y_pred)**2))**(1/2)


In [ ]:
aux_rmse_u,aux_endp_u = [], []
for i in range(50):   
    y_pred = [10] # given a huge constant  
    train_set, test_set = train_test_sep(df_u, 1)
    while (abs(y_pred[0] - float(test_set['transmission_rate']))) > 0.05: 
        # choose a number of time steps
        n_steps_in, n_steps_out = 2, 1

        # split into samples
        train_tr, test_tr = split_sequence(train_set['transmission_rate'],n_steps_in, n_steps_out)
        train_cfr, test_cfr = split_sequence(train_set['case_fatality_rate'],n_steps_in, n_steps_out)
        train_pfv, test_pfv = split_sequence(train_set['people_fully_vaccinated'],n_steps_in, n_steps_out)
        train_aux = np.append(train_tr, train_cfr, axis=1)
        X = np.append(train_aux, train_pfv, axis=1)
        y = test_tr

        # summarize the data
        # for i in range(len(X)):
        #print(X[i], y[i])


        # reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # define model
        model = Sequential()
        model.add(Bidirectional(LSTM(200, activation='relu'), input_shape=(n_steps_in*3, n_features)))
        model.add(Dropout(0.2))
        model.add(Dense(n_steps_out))
        model.compile(optimizer='adam', loss='mse')

        # weights = model.layers[0].get_weights()[0]
        # biases = model.layers[0].get_weights()[1]
        # fit model
        model.fit(X, y, epochs=200, verbose=0, validation_split=0.05)
        # prediction for RMSE
        t_set = list(train_set['transmission_rate'])
        cfr = train_set['case_fatality_rate'][(len(t_set) -n_steps_in):len(t_set)]
        pfv = train_set['people_fully_vaccinated'][(len(t_set) -n_steps_in):len(t_set)]
        x_input = t_set[(len(t_set) -n_steps_in):len(t_set)]
        y_pred = predict(model, x_input,cfr,pfv, n_steps_in*3, n_features, 1)
    # prediction for Endpoints
    t_set = list(df_u['transmission_rate'])
    x_input = t_set[(len(t_set) -n_steps_in):len(t_set)]
    y_predend = predict_end_u(model, x_input, n_steps_in*3, n_features, 48)
    print(i, y_pred)
    aux_endp_u.append(count_endpoint(y_predend))
    aux_rmse_u.append(y_pred)


In [ ]:
aux_rmse_s,aux_endp_s = [], []
for i in range(50):   
    y_pred = [10] 
    train_set, test_set = train_test_sep(df_s, 1)
    while (abs(y_pred[0] - float(test_set['transmission_rate']))) > 0.05: 
        # choose a number of time steps
        n_steps_in, n_steps_out = 2, 1

        # split into samples
        train_tr, test_tr = split_sequence(train_set['transmission_rate'],n_steps_in, n_steps_out)
        train_cfr, test_cfr = split_sequence(train_set['case_fatality_rate'],n_steps_in, n_steps_out)
        train_pfv, test_pfv = split_sequence(train_set['people_fully_vaccinated'],n_steps_in, n_steps_out)
        train_aux = np.append(train_tr, train_cfr, axis=1)
        X = np.append(train_aux, train_pfv, axis=1)
        y = test_tr

        # summarize the data
        # for i in range(len(X)):
        #print(X[i], y[i])


        # reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # define model
        model = Sequential()
        model.add(Bidirectional(LSTM(200, activation='relu'), input_shape=(n_steps_in*3, n_features)))
        model.add(Dropout(0.1))
        model.add(Dense(n_steps_out))
        model.compile(optimizer='adam', loss='mse')

        # weights = model.layers[0].get_weights()[0]
        # biases = model.layers[0].get_weights()[1]
        # fit model
        model.fit(X, y, epochs=200, verbose=0, validation_split=0.05)
        # prediction for RMSE
        t_set = list(train_set['transmission_rate'])
        cfr = train_set['case_fatality_rate'][(len(t_set) -n_steps_in):len(t_set)]
        pfv = train_set['people_fully_vaccinated'][(len(t_set) -n_steps_in):len(t_set)]
        x_input = t_set[(len(t_set) -n_steps_in):len(t_set)]
        y_pred = predict(model, x_input,cfr,pfv, n_steps_in*3, n_features, 1)
    # prediction for Endpoints
    t_set = list(df_u['transmission_rate'])
    x_input = t_set[(len(t_set) -n_steps_in):len(t_set)]
    y_predend = predict_end_s(model, x_input, n_steps_in*3, n_features, 48)
    print(i, y_pred)
    aux_endp_s.append(count_endpoint(y_predend))
    aux_rmse_s.append(y_pred)


In [ ]:
aux_rmse_c,aux_endp_c = [], []
for i in range(50):   
    y_pred = [10]
    train_set, test_set = train_test_sep(df_c, 1)
    
    while (abs(y_pred[0] - float(test_set['transmission_rate']))) > 0.05: 
        # choose a number of time steps
        n_steps_in, n_steps_out = 2, 1

        # split into samples
        train_tr, test_tr = split_sequence(train_set['transmission_rate'],n_steps_in, n_steps_out)
        train_cfr, test_cfr = split_sequence(train_set['case_fatality_rate'],n_steps_in, n_steps_out)
        train_pfv, test_pfv = split_sequence(train_set['people_fully_vaccinated'],n_steps_in, n_steps_out)
        train_aux = np.append(train_tr, train_cfr, axis=1)
        X = np.append(train_aux, train_pfv, axis=1)
        y = test_tr

        # summarize the data
        # for i in range(len(X)):
        #print(X[i], y[i])


        # reshape from [samples, timesteps] into [samples, timesteps, features]
        n_features = 1
        X = X.reshape((X.shape[0], X.shape[1], n_features))

        # define model
        model = Sequential()
        model.add(Bidirectional(LSTM(200, activation='relu'), input_shape=(n_steps_in*3, n_features)))
        model.add(Dropout(0.2))
        model.add(Dense(n_steps_out))
        model.compile(optimizer='adam', loss='mse')

        # weights = model.layers[0].get_weights()[0]
        # biases = model.layers[0].get_weights()[1]
        # fit model
        model.fit(X, y, epochs=200, verbose=0, validation_split=0.05)
        # prediction for RMSE
        t_set = list(train_set['transmission_rate'])
        cfr = train_set['case_fatality_rate'][(len(t_set) -n_steps_in):len(t_set)]
        pfv = train_set['people_fully_vaccinated'][(len(t_set) -n_steps_in):len(t_set)]
        x_input = t_set[(len(t_set) -n_steps_in):len(t_set)]
        y_pred = predict(model, x_input,cfr,pfv, n_steps_in*3, n_features, 1)
    # prediction for Endpoints
    t_set = list(df_u['transmission_rate'])
    x_input = t_set[(len(t_set) -n_steps_in):len(t_set)]
    y_predend = predict_end_c(model, x_input, n_steps_in*3, n_features, 48)
    print(i, y_pred)
    aux_endp_c.append(count_endpoint(y_predend))
    aux_rmse_c.append(y_pred)

In [18]:
fore_s = {'RMSE':aux_rmse_s,
        'End Date':aux_endp_s,
        'Country':['Spain' for i in range(len(aux_rmse_s))]
           }
fore_c = {'RMSE':aux_rmse_c,
        'End Date':aux_endp_c,
        'Country':['Canada' for i in range(len(aux_rmse_s))]
           }
fore_u = {'RMSE':aux_rmse_u,
        'End Date':aux_endp_u,
        'Country':['US' for i in range(len(aux_rmse_s))]
           }
f_s = pd.DataFrame(fore_s)
f_c = pd.DataFrame(fore_c)
f_u = pd.DataFrame(fore_u)
df_a = pd.concat([f_u, f_c])
df_p = pd.concat([df_a, f_s]) 
df_p['RMSE'] = [abs(i[0]) for i in df_p['RMSE']] 
df_p.to_csv('result_data.csv',index=False)